In [ ]:
import warnings
import os
import sys
import re
import biolord
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import sklearn
import torch


from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mutual_info_score
)

In [ ]:
# Setup the AnnData

adata = sc.read("/work/Biolord_all/new_data_fixed_normalized/HTAPP_997_processed_normalized_FINAL.h5ad")

In [ ]:
adata

In [ ]:
# Subset the data
subset_mask = (adata.obs['replicate'] == '1') & (adata.obs['cell_type'] == 'hepatocyte')
subset_adata = adata[subset_mask].copy()  
train_adata = adata[~subset_mask].copy()  # Training data is everything except the subset

In [ ]:
print(train_adata.obs.groupby(['replicate', 'cell_type']).size())

In [ ]:
adata.obs["replicate"]

In [ ]:
adata.obs["cell_type"]

In [ ]:
biolord.Biolord.setup_anndata(
    train_adata, ordered_attributes_keys=None, categorical_attributes_keys=["cell_type", "Phase", "replicate", "compartments", "cnv_pass_mal"]
)

In [ ]:
# Instantiate a Biolord model

module_params = {
    "decoder_width": 1024,
    "decoder_depth": 4,
    "attribute_nn_width": 512,
    "attribute_nn_depth": 2,
    "n_latent_attribute_categorical": 4,
    "gene_likelihood": "normal",
    "reconstruction_penalty": 1e2,
    "unknown_attribute_penalty": 1e1,
    "unknown_attribute_noise_param": 1e-1,
    "attribute_dropout_rate": 0.1,
    "use_batch_norm": False,
    "use_layer_norm": False,
    "seed": 42,
}

In [ ]:
model = biolord.Biolord(
    adata=train_adata,
    n_latent=32,
    model_name="HTAPP_Biolord_LOA_run_fix_0905",
    module_params=module_params,
    train_classifiers=False
)

In [ ]:
# Train the model

trainer_params = {
    "n_epochs_warmup": 0,
    "latent_lr": 1e-4,
    "latent_wd": 1e-4,
    "decoder_lr": 1e-4,
    "decoder_wd": 1e-4,
    "attribute_nn_lr": 1e-2,
    "attribute_nn_wd": 4e-8,
    "step_size_lr": 45,
    "cosine_scheduler": True,
    "scheduler_final_lr": 1e-5,
}

In [ ]:
model.train(
    max_epochs=500,
    batch_size=512,
    plan_kwargs=trainer_params,
    early_stopping=True,
    early_stopping_patience=20,
    check_val_every_n_epoch=10,
    enable_checkpointing=False,
)

In [ ]:
# Save the model
model.save(dir_path="HTAP_BIOLORD/HTAP_BIOLORD/LOA_HTAPP_final" + f"{model.model_name}_model/")

In [ ]:
model = model.load("/work/Biolord_all/HTAP_BIOLORD/HTAP_BIOLORD/HTAP_BIOLORD/LOA_HTAPP_fixed_dataHTAPP_Biolord_LOA_run_fix_0905_model", adata = train_adata)

In [ ]:
model

In [ ]:
size = 4
vals = ["generative_mean_accuracy", "generative_var_accuracy", "biolord_metric"]
fig, axs = plt.subplots(nrows=1, ncols=len(vals), figsize=(size * len(vals), size))

model.epoch_history = pd.DataFrame().from_dict(model.training_plan.epoch_history)
for i, val in enumerate(vals):
    sns.lineplot(
        x="epoch",
        y=val,
        hue="mode",
        data=model.epoch_history[model.epoch_history["mode"] == "valid"],
        ax=axs[i],
    )

plt.tight_layout()
plt.show()

In [ ]:
rep_2_base = train_adata[
    (train_adata.obs["replicate"] == "2") &
    (train_adata.obs["cell_type"] == "hepatocyte")
].copy()

In [ ]:
rep_2_base.obs["replicate"]

In [ ]:
rec_rep_2_base,_ = model.predict(rep_2_base, batch_size=256)

In [ ]:
cf = rep_2_base.copy()
cf.obs["replicate"] = "1"

In [ ]:
cf.obs["replicate"]

In [ ]:
cf_pred,_ = model.predict(cf, batch_size=256)

In [ ]:
# Compute per-gene ground-truth means
# Gt for females is now females with hepatocytes

rep2_gt = (
    rep_2_base.X.toarray().mean(axis=0)
    if hasattr(rep_2_base.X, "toarray")
    else rep_2_base.X.mean(axis=0)
)


#    Gt for the male is now the held out dataset
rep1_gt = (
    subset_adata.X.toarray().mean(axis=0)
    if hasattr(subset_adata.X, "toarray")
    else subset_adata.X.mean(axis=0)
)

In [ ]:
#  Compute per-gene prediction means 
baseline_pred = np.asarray(rec_rep_2_base.X.mean(axis=0)).ravel()
cf_pred       = np.asarray(cf_pred.X.mean(axis=0)).ravel()

In [ ]:
f_m = np.sqrt(np.mean((baseline_pred - rep1_gt)**2)) #pred-F --- TM
m_m = np.sqrt(np.mean((cf_pred - rep1_gt)**2)) # pred_M --- TM
f_f  =np.sqrt(np.mean((baseline_pred - rep2_gt)**2)) # pref F ----- TF
m_f = np.sqrt(np.mean((cf_pred - rep2_gt)**2)) # pref M ---- TF

print(f"RMSE pred F --- TM: {f_m:.4f}")
print(f"RMSE pred M --- TM (counterfactual)  : {m_m:.4f}")
print(f"RMSE pred F ----- TF : {f_f:.4f}")
print(f"RMSE pred M ---- TF (counterfactual)  : {m_f:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Define RMSE values
rmse_data = [
    ["0.0211", "0.0266"],  # True Female
    ["0.0477", "0.1101"],  # True Male
]

# Define labels
column_labels = ["Pred Replicate 2", "Pred Replicate 1 (CF)"]
row_labels = ["True Replicate 2", "True Replicate 1"]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 2))
ax.axis('tight')
ax.axis('off')

# Create the table
table = ax.table(
    cellText=rmse_data,
    rowLabels=row_labels,
    colLabels=column_labels,
    cellLoc='center',
    loc='center'
)

table.scale(1, 2)  # Increase row height
table.auto_set_font_size(False)
table.set_fontsize(12)

# Color the lower row cells
# Note: Rows and columns are 1-indexed in table.get_celld()
table[(2, 0)].set_facecolor("orange")  # True Male, Pred Female
table[(2, 1)].set_facecolor("lightblue")  # True Male, Pred Male

plt.title("RMSE between Predictions and Ground truth", pad=20)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Log-transform the data
log_male_gt = np.log1p(rep1_gt)
log_cf_pred = np.log1p(cf_pred)
log_baseline_pred = np.log1p(baseline_pred)

# Scatter plot of log-transformed ground truth vs. predictions
fig, ax = plt.subplots(figsize=(10, 6))

sns.scatterplot(
    x=log_male_gt, y=log_cf_pred,
    alpha=0.7, ax=ax
)

sns.scatterplot(
    x=log_male_gt, y=log_baseline_pred,
    alpha=0.7, ax=ax
)

# Identity line (in log space)
mn = min(log_male_gt.min(), log_male_gt.min())
mx = max(log_male_gt.max(), log_male_gt.max())
ax.plot([mn, mx], [mn, mx], ls="--", color="red")


ax.set_xlabel("Log Ground Truth (Male)")
ax.set_ylabel("Log Predicted Mean Expression")
ax.set_title("Leave One Out analysis" )
ax.legend(loc="upper left")
plt.tight_layout()
plt.show()
